In [41]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import matplotlib.pyplot as plt

from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV


In [42]:
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [43]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

In [44]:
def build_features(features, data, isForTrain = True):
    features.extend(['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday'])
    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    
    features.extend(['DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear'])
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    if isForTrain:
        data['Sales'] = data.Sales
    
    # CompetitionOpenSinceYear 和 CompetitionOpenSinceMonth为空使用平均值
    features.append('CompetitionOpen')
    
    data.loc[data.CompetitionOpenSinceYear == 0, 'CompetitionOpenSinceYear'] = data.loc[data.CompetitionOpenSinceYear != 0, 'CompetitionOpenSinceYear'].mean()
    data.loc[data.CompetitionOpenSinceMonth == 0, 'CompetitionOpenSinceMonth'] = data.loc[data.CompetitionOpenSinceMonth != 0, 'CompetitionOpenSinceMonth'].mean()
#     data.loc[data.CompetitionOpenSinceYear == 0, 'CompetitionOpenSinceYear'] = data.loc[data.CompetitionOpenSinceYear != 0, 'CompetitionOpenSinceYear'].median()
#     data.loc[data.CompetitionOpenSinceMonth == 0, 'CompetitionOpenSinceMonth'] = data.loc[data.CompetitionOpenSinceMonth != 0, 'CompetitionOpenSinceMonth'].median()
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + (data.Month - data.CompetitionOpenSinceMonth)
    
    features.append('PromoOpen')
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) +  (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
#     data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    data.loc[data.Promo2SinceYear == 0, 'PromoOpen'] = 0
    
    features.append('IsPromoMonth')
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1
    
    features.append('SalesPerDay')
    features.append('CustomersPerDay')
    
    if isForTrain:
        for store in data.Store.unique():
            data.loc[data.Store == store, 'SalesPerDay'] = data.loc[data.Store == store, 'Sales'].mean()
            data.loc[data.Store == store, 'CustomersPerDay'] = data.loc[data.Store == store, 'Customers'].mean()
    if isForTrain:
        data = data.drop('Sales', 1)
    
    return data


In [50]:
def train_model(params, features, train):
    num_boost_round = 20000

    print("Train a XGBoost model")
    X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)
    y_train = np.log1p(X_train.Sales)
    y_valid = np.log1p(X_valid.Sales)
    dtrain = xgb.DMatrix(X_train[features], y_train)
    dvalid = xgb.DMatrix(X_valid[features], y_valid)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
    return gbm

In [46]:
# def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
#         alg.set_params(n_estimators=cvresult.shape[0])

#     #Fit the algorithm on the data
#     alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')

#     #Predict training set:
#     dtrain_predictions = alg.predict(dtrain[predictors])
#     dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

#     #Print model report:
#     print "\nModel Report"
#     print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
#     print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)

#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

In [47]:
# importing train data to learn
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("./dataset/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("./dataset/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("./dataset/store.csv")

train.fillna(1, inplace=True)
test.fillna(1, inplace=True)

train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')
features = []
build_features(features, train, True)
build_features([], test, False)
for store in train.Store.unique():
    test.loc[test.Store == store, 'SalesPerDay'] = train.loc[train.Store == store, 'Sales'].mean()
    test.loc[test.Store == store, 'CustomersPerDay'] = train.loc[train.Store == store, 'Customers'].mean()




In [51]:
params = {"objective": "reg:linear",
              "booster" : "gbtree",
              "eta": 0.02,
              "max_depth": 8,
              "subsample": 0.9,
              "colsample_bytree": 0.7,
              "silent": 1,
              "seed": 0
              }
gbm = train_model(params, features, train)
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_submission.csv", index=False)

Train a XGBoost model
[0]	train-rmse:8.10368	eval-rmse:8.10236	train-rmspe:0.999837	eval-rmspe:0.999837
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:7.94173	eval-rmse:7.94042	train-rmspe:0.999779	eval-rmspe:0.999779
[2]	train-rmse:7.78306	eval-rmse:7.78172	train-rmspe:0.999713	eval-rmspe:0.999712
[3]	train-rmse:7.62751	eval-rmse:7.6262	train-rmspe:0.999637	eval-rmspe:0.999636
[4]	train-rmse:7.47508	eval-rmse:7.47379	train-rmspe:0.99955	eval-rmspe:0.99955
[5]	train-rmse:7.32571	eval-rmse:7.32442	train-rmspe:0.999452	eval-rmspe:0.999451
[6]	train-rmse:7.17936	eval-rmse:7.17806	train-rmspe:0.99934	eval-rmspe:0.999339
[7]	train-rmse:7.03594	eval-rmse:7.03459	train-rmspe:0.999214	eval-rmspe:0.999212
[8]	train-rmse:6.89543	eval-rmse:6.89406	train-rmspe:0.999071	eval-rmspe:0.999069
[9]	train-rmse:6.7577	eval-rmse:6.75631	train-rmspe:0.998911	eval-rmspe:0.998909
[10]	train-rmse:6

[98]	train-rmse:1.13324	eval-rmse:1.13233	train-rmspe:0.672234	eval-rmspe:0.670344
[99]	train-rmse:1.11105	eval-rmse:1.11014	train-rmspe:0.665021	eval-rmspe:0.663028
[100]	train-rmse:1.08933	eval-rmse:1.08841	train-rmspe:0.657812	eval-rmspe:0.655705
[101]	train-rmse:1.06808	eval-rmse:1.06717	train-rmspe:0.650586	eval-rmspe:0.648367
[102]	train-rmse:1.04723	eval-rmse:1.04631	train-rmspe:0.643372	eval-rmspe:0.641036
[103]	train-rmse:1.02684	eval-rmse:1.0259	train-rmspe:0.636161	eval-rmspe:0.633696
[104]	train-rmse:1.00687	eval-rmse:1.00594	train-rmspe:0.628951	eval-rmspe:0.626363
[105]	train-rmse:0.987254	eval-rmse:0.98632	train-rmspe:0.621769	eval-rmspe:0.619049
[106]	train-rmse:0.968031	eval-rmse:0.967087	train-rmspe:0.614602	eval-rmspe:0.611745
[107]	train-rmse:0.949214	eval-rmse:0.948264	train-rmspe:0.607455	eval-rmspe:0.604455
[108]	train-rmse:0.930779	eval-rmse:0.929811	train-rmspe:0.600325	eval-rmspe:0.597178
[109]	train-rmse:0.912748	eval-rmse:0.911755	train-rmspe:0.593205	eval-r

[194]	train-rmse:0.220069	eval-rmse:0.218097	train-rmspe:0.231523	eval-rmspe:0.194623
[195]	train-rmse:0.217594	eval-rmse:0.215614	train-rmspe:0.230253	eval-rmspe:0.192863
[196]	train-rmse:0.215206	eval-rmse:0.213213	train-rmspe:0.229046	eval-rmspe:0.191181
[197]	train-rmse:0.212907	eval-rmse:0.210906	train-rmspe:0.227909	eval-rmspe:0.189563
[198]	train-rmse:0.210647	eval-rmse:0.208632	train-rmspe:0.226796	eval-rmspe:0.187979
[199]	train-rmse:0.208411	eval-rmse:0.206385	train-rmspe:0.225675	eval-rmspe:0.186341
[200]	train-rmse:0.206275	eval-rmse:0.204226	train-rmspe:0.224595	eval-rmspe:0.184824
[201]	train-rmse:0.204165	eval-rmse:0.202103	train-rmspe:0.223496	eval-rmspe:0.183334
[202]	train-rmse:0.202159	eval-rmse:0.200085	train-rmspe:0.222549	eval-rmspe:0.18194
[203]	train-rmse:0.200215	eval-rmse:0.19813	train-rmspe:0.221633	eval-rmspe:0.180593
[204]	train-rmse:0.198286	eval-rmse:0.196187	train-rmspe:0.220724	eval-rmspe:0.179209
[205]	train-rmse:0.196452	eval-rmse:0.194347	train-rmspe

[290]	train-rmse:0.140163	eval-rmse:0.138193	train-rmspe:0.200861	eval-rmspe:0.144442
[291]	train-rmse:0.140027	eval-rmse:0.138071	train-rmspe:0.200859	eval-rmspe:0.144417
[292]	train-rmse:0.139807	eval-rmse:0.137863	train-rmspe:0.200817	eval-rmspe:0.144263
[293]	train-rmse:0.139687	eval-rmse:0.137749	train-rmspe:0.200839	eval-rmspe:0.144243
[294]	train-rmse:0.139586	eval-rmse:0.137652	train-rmspe:0.200864	eval-rmspe:0.144244
[295]	train-rmse:0.139468	eval-rmse:0.137539	train-rmspe:0.200884	eval-rmspe:0.144219
[296]	train-rmse:0.139341	eval-rmse:0.137416	train-rmspe:0.200781	eval-rmspe:0.14419
[297]	train-rmse:0.139184	eval-rmse:0.137276	train-rmspe:0.200737	eval-rmspe:0.144127
[298]	train-rmse:0.139052	eval-rmse:0.137147	train-rmspe:0.200725	eval-rmspe:0.144091
[299]	train-rmse:0.138937	eval-rmse:0.137034	train-rmspe:0.20074	eval-rmspe:0.144078
[300]	train-rmse:0.13881	eval-rmse:0.136915	train-rmspe:0.20073	eval-rmspe:0.144046
[301]	train-rmse:0.138651	eval-rmse:0.136772	train-rmspe:0

[386]	train-rmse:0.1312	eval-rmse:0.129868	train-rmspe:0.193498	eval-rmspe:0.139378
[387]	train-rmse:0.131125	eval-rmse:0.129797	train-rmspe:0.193124	eval-rmspe:0.139323
[388]	train-rmse:0.131079	eval-rmse:0.129751	train-rmspe:0.193098	eval-rmspe:0.139286
[389]	train-rmse:0.131013	eval-rmse:0.129692	train-rmspe:0.193001	eval-rmspe:0.139237
[390]	train-rmse:0.130952	eval-rmse:0.129631	train-rmspe:0.192977	eval-rmspe:0.139189
[391]	train-rmse:0.130899	eval-rmse:0.129585	train-rmspe:0.192875	eval-rmspe:0.139138
[392]	train-rmse:0.130849	eval-rmse:0.129533	train-rmspe:0.1928	eval-rmspe:0.139093
[393]	train-rmse:0.130781	eval-rmse:0.129469	train-rmspe:0.19274	eval-rmspe:0.139031
[394]	train-rmse:0.130739	eval-rmse:0.129429	train-rmspe:0.19268	eval-rmspe:0.139006
[395]	train-rmse:0.130677	eval-rmse:0.129368	train-rmspe:0.192643	eval-rmspe:0.138959
[396]	train-rmse:0.130618	eval-rmse:0.129305	train-rmspe:0.192486	eval-rmspe:0.138896
[397]	train-rmse:0.130587	eval-rmse:0.129275	train-rmspe:0.1

...

[15722]	train-rmse:0.066136	eval-rmse:0.08292	train-rmspe:0.072077	eval-rmspe:0.087563
[15723]	train-rmse:0.066135	eval-rmse:0.082919	train-rmspe:0.072076	eval-rmspe:0.087563
[15724]	train-rmse:0.066134	eval-rmse:0.082918	train-rmspe:0.072075	eval-rmspe:0.087562
[15725]	train-rmse:0.066133	eval-rmse:0.082918	train-rmspe:0.072074	eval-rmspe:0.087561
[15726]	train-rmse:0.066132	eval-rmse:0.082918	train-rmspe:0.072072	eval-rmspe:0.087561
[15727]	train-rmse:0.066131	eval-rmse:0.082918	train-rmspe:0.072072	eval-rmspe:0.087561
[15728]	train-rmse:0.066131	eval-rmse:0.082917	train-rmspe:0.072071	eval-rmspe:0.08756
[15729]	train-rmse:0.066129	eval-rmse:0.082918	train-rmspe:0.072056	eval-rmspe:0.087561
[15730]	train-rmse:0.066128	eval-rmse:0.082918	train-rmspe:0.072056	eval-rmspe:0.087561
[15731]	train-rmse:0.066127	eval-rmse:0.082918	train-rmspe:0.072055	eval-rmspe:0.087562
[15732]	train-rmse:0.066126	eval-rmse:0.082917	train-rmspe:0.072054	eval-rmspe:0.08756
[15733]	train-rmse:0.066126	eval-rm

[15816]	train-rmse:0.066053	eval-rmse:0.082908	train-rmspe:0.071929	eval-rmspe:0.087553
[15817]	train-rmse:0.066052	eval-rmse:0.082908	train-rmspe:0.071928	eval-rmspe:0.087552
[15818]	train-rmse:0.066051	eval-rmse:0.082909	train-rmspe:0.071927	eval-rmspe:0.087553
[15819]	train-rmse:0.06605	eval-rmse:0.082909	train-rmspe:0.071926	eval-rmspe:0.087553
[15820]	train-rmse:0.06605	eval-rmse:0.082909	train-rmspe:0.071926	eval-rmspe:0.087553
[15821]	train-rmse:0.066049	eval-rmse:0.082909	train-rmspe:0.071925	eval-rmspe:0.087553
[15822]	train-rmse:0.066048	eval-rmse:0.08291	train-rmspe:0.071924	eval-rmspe:0.087554
[15823]	train-rmse:0.066046	eval-rmse:0.082909	train-rmspe:0.071918	eval-rmspe:0.087553
[15824]	train-rmse:0.066045	eval-rmse:0.082909	train-rmspe:0.071917	eval-rmspe:0.087553
[15825]	train-rmse:0.066044	eval-rmse:0.082909	train-rmspe:0.071916	eval-rmspe:0.087553
[15826]	train-rmse:0.066043	eval-rmse:0.082909	train-rmspe:0.071915	eval-rmspe:0.087553
[15827]	train-rmse:0.066042	eval-rm

[15910]	train-rmse:0.065965	eval-rmse:0.082892	train-rmspe:0.071719	eval-rmspe:0.087534
[15911]	train-rmse:0.065963	eval-rmse:0.082892	train-rmspe:0.071716	eval-rmspe:0.087534
[15912]	train-rmse:0.065962	eval-rmse:0.082891	train-rmspe:0.071714	eval-rmspe:0.087533
[15913]	train-rmse:0.065962	eval-rmse:0.082891	train-rmspe:0.071714	eval-rmspe:0.087533
[15914]	train-rmse:0.065961	eval-rmse:0.082891	train-rmspe:0.071713	eval-rmspe:0.087533
[15915]	train-rmse:0.065959	eval-rmse:0.082891	train-rmspe:0.071709	eval-rmspe:0.087533
[15916]	train-rmse:0.065958	eval-rmse:0.082891	train-rmspe:0.071709	eval-rmspe:0.087533
[15917]	train-rmse:0.065957	eval-rmse:0.082892	train-rmspe:0.071707	eval-rmspe:0.087533
[15918]	train-rmse:0.065956	eval-rmse:0.082892	train-rmspe:0.071706	eval-rmspe:0.087534
[15919]	train-rmse:0.065955	eval-rmse:0.082893	train-rmspe:0.071705	eval-rmspe:0.087533
[15920]	train-rmse:0.065954	eval-rmse:0.082892	train-rmspe:0.071704	eval-rmspe:0.087533
[15921]	train-rmse:0.065953	eval

[16004]	train-rmse:0.065871	eval-rmse:0.082877	train-rmspe:0.071599	eval-rmspe:0.087518
[16005]	train-rmse:0.06587	eval-rmse:0.082876	train-rmspe:0.071598	eval-rmspe:0.087518
[16006]	train-rmse:0.065869	eval-rmse:0.082875	train-rmspe:0.071597	eval-rmspe:0.087517
[16007]	train-rmse:0.065869	eval-rmse:0.082875	train-rmspe:0.071597	eval-rmspe:0.087517
[16008]	train-rmse:0.065867	eval-rmse:0.082875	train-rmspe:0.071594	eval-rmspe:0.087514
[16009]	train-rmse:0.065866	eval-rmse:0.082875	train-rmspe:0.071594	eval-rmspe:0.087514
[16010]	train-rmse:0.065865	eval-rmse:0.082875	train-rmspe:0.071593	eval-rmspe:0.087514
[16011]	train-rmse:0.065864	eval-rmse:0.082875	train-rmspe:0.071591	eval-rmspe:0.087514
[16012]	train-rmse:0.065863	eval-rmse:0.082875	train-rmspe:0.07159	eval-rmspe:0.087514
[16013]	train-rmse:0.065862	eval-rmse:0.082875	train-rmspe:0.071589	eval-rmspe:0.087514
[16014]	train-rmse:0.065862	eval-rmse:0.082875	train-rmspe:0.071589	eval-rmspe:0.087514
[16015]	train-rmse:0.065861	eval-r

[16098]	train-rmse:0.065786	eval-rmse:0.082853	train-rmspe:0.071491	eval-rmspe:0.087489
[16099]	train-rmse:0.065785	eval-rmse:0.082854	train-rmspe:0.071488	eval-rmspe:0.08749
[16100]	train-rmse:0.065784	eval-rmse:0.082854	train-rmspe:0.071486	eval-rmspe:0.087492
[16101]	train-rmse:0.065782	eval-rmse:0.082855	train-rmspe:0.071484	eval-rmspe:0.087492
[16102]	train-rmse:0.065781	eval-rmse:0.082855	train-rmspe:0.071483	eval-rmspe:0.087492
[16103]	train-rmse:0.06578	eval-rmse:0.082854	train-rmspe:0.071483	eval-rmspe:0.087492
[16104]	train-rmse:0.065779	eval-rmse:0.082854	train-rmspe:0.071482	eval-rmspe:0.087492
[16105]	train-rmse:0.065779	eval-rmse:0.082854	train-rmspe:0.071482	eval-rmspe:0.087492
[16106]	train-rmse:0.065778	eval-rmse:0.082854	train-rmspe:0.071479	eval-rmspe:0.087492
[16107]	train-rmse:0.065777	eval-rmse:0.082854	train-rmspe:0.071478	eval-rmspe:0.087492
[16108]	train-rmse:0.065776	eval-rmse:0.082854	train-rmspe:0.071477	eval-rmspe:0.087492
[16109]	train-rmse:0.065775	eval-r

In [52]:
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_submission.csv", index=False)
# # learn rate
# dtest = xgb.DMatrix(test[features])
# for depth in range(6):
#     for eta in range(20):
#         print("xgboost_max_depth="+str(5 + depth) + "eta="+str(0.01 * (eta + 1)) +"_submission.csv")
#         params = {"objective": "reg:linear",
#               "booster" : "gbtree",
#               "eta": 0.01 * (eta + 1),
#               "max_depth": 5 + depth,
#               "subsample": 0.9,
#               "colsample_bytree": 0.7,
#               "silent": 1,
#               "seed": 0
#               }
#         gbm = train_model(params, features, train)

#         test_probs = gbm.predict(dtest)
#         result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
#         result.to_csv("xgboost_max_depth="+str(5 + depth) + "eta="+str(0.01 * (eta + 1)) +"_submission.csv", index=False)

In [ ]:
# train-rmse:0.065806	eval-rmse:0.082855	train-rmspe:0.071516	eval-rmspe:0.087489